In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql import Row

In [2]:
try:
    sc.stop()
except:
    print("SparkContext has not been initiated")

SparkContext has not been initiated


In [4]:
conf = (SparkConf() 
    .set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,org.postgresql:postgresql:42.5.0")
    .setMaster("local") #spark://spark-master:7077
    .setAppName('SparkApp') 
)

sc = SparkContext(conf=conf)

spark = SparkSession(sc)

spark

# OLD
# conf = SparkConf() \
#     .set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector:10.0.0,org.postgresql:postgresql:42.5.0")

# sc = SparkContext(conf=conf)
# # sql_context = SQLContext(sc)

# spark = (SparkSession.builder
#     .master("spark://spark-master:7077")
#     .appName("myApp")
#     .getOrCreate()
# )
# spark

# Load dataframe

In [5]:
player_df = spark.read \
    .format("com.mongodb.spark.sql.DefaultSource") \
    .option('uri', "mongodb://mongodb:27017/football_data_new.player") \
    .load()
    
player_df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- info: struct (nullable = true)
 |    |-- ShortName: string (nullable = true)
 |    |-- FullName: string (nullable = true)
 |    |-- Position: string (nullable = true)
 |    |-- Height: string (nullable = true)
 |    |-- Weight: string (nullable = true)
 |    |-- Footed: string (nullable = true)
 |    |-- DOB: string (nullable = true)
 |    |-- Nationality: string (nullable = true)
 |    |-- Club: string (nullable = true)
 |-- stats: struct (nullable = true)
 |    |-- adv_goalkeeping: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- #OPA: integer (nullable = true)
 |    |    |    |-- #OPA/90: double (nullable = true)
 |    |    |    |-- /90: double (nullable = true)
 |    |    |    |-- 90s: double (nullable = true)
 |    |    |    |-- Age: integer (nullable = true)
 |    |    |    |-- Att: integer (nullable = true)
 |    |    |    |-- AvgDist: double (nullable = true)
 |    |    |    |-- AvgLen:

# Get schema

In [6]:
info_table_schema = player_df.select('info').schema[0].dataType

In [7]:
stats_table_names = player_df.schema['stats'].dataType.fieldNames()

stats_table_schemas = {}
for table_name in stats_table_names:
    full_table_name = 'stats.' + table_name
    table_schema = player_df.select(full_table_name).schema[0].dataType.elementType
    stats_table_schemas[table_name] = table_schema

# Process table

In [8]:
from tqdm import tqdm

In [9]:
stats_tables = {}
info_table = spark.createDataFrame(
    data = sc.emptyRDD(),
    schema = info_table_schema
)

for player_row in tqdm(player_df.collect()):
    player_id = player_row['_id']
    player_info = player_row['info']
    player_stats = player_row['stats']
    
    # Parse Info table
    try:
        info_df = spark.createDataFrame([player_info], schema=info_table_schema) 
        info_table = info_table.union(info_df)
    except Exception as e: 
        print(e)
        print("ERROR when parsing info", player_id, player_info)
    
    # Parse Stats tables
    for table_name in stats_table_names:
        table_data = player_stats[table_name]
        if not table_data:
            continue

        table_schema = stats_table_schemas[table_name]

        try:
            sub_stats_df = spark.createDataFrame(table_data, schema=table_schema) 
            sub_stats_df = sub_stats_df.withColumn("PlayerID", lit(player_id))

            if table_name in stats_tables:
                stats_tables[table_name] = stats_tables[table_name].union(sub_stats_df)
            else:
                stats_tables[table_name] = sub_stats_df    
        except Exception as e: 
            print(e)
            print("ERROR when parsing stats", player_id, table_name)

100%|██████████| 710/710 [13:38<00:00,  1.15s/it]


## Process Info

In [ ]:
import

In [10]:
info_table.printSchema()

root
 |-- ShortName: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Footed: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Club: string (nullable = true)



In [14]:
info_table.collect()[0]

Row(ShortName='Nicolas Pépé', FullName=None, Position=' FW-MF (AM, right)\xa0▪\xa0 ', Height='178cm', Weight='68kg', Footed=' Left', DOB='1995-05-29', Nationality='CIV', Club='132ebc33')

In [ ]:
def process_text(text):
    if not isinstance(text, str):
        return text

info_table.withColumn("ShortName", process_text)

## Process Stats

In [ ]:
for table_name in stats_tables:
    table_data = stats_tables[table_name]
    print(table_name)
    table_data.show(n=5)
    print()

# Save table

In [ ]:
(info_table.write.format("jdbc")
    .option("url", "jdbc:postgresql://postgres:5432/news_crawled")
    .option("driver", "org.postgresql.Driver")
    .option("dbtable", "player_info")
    .option("user", "test1")
    .option("password", "test1")
    .save()
)

In [ ]:
for table_name in stats_tables:
    full_table_name = 'player_' + table_name
    table_df = stats_tables[table_name]
    
    (table_df.write.format("jdbc")
        .option("url", "jdbc:postgresql://postgres:5432/news_crawled")
        .option("driver", "org.postgresql.Driver")
        .option("dbtable", full_table_name)
        .option("user", "test1")
        .option("password", "test1")
        .save()
    )
    